<a href="https://colab.research.google.com/github/strucker-eth/Arrow-funtions-Js-Solidity/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Install spark**

In [508]:
#Install JDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [509]:
#Install Pyspark
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

# Unzip the file
!tar xf spark-3.3.2-bin-hadoop3.tgz


In [510]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"


In [529]:
!pip install findspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [530]:
import findspark
findspark.init('/content/spark-3.3.2-bin-hadoop3')

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [564]:
'''
load models
'''
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics


# **2. Load dataset**

In [552]:
'''
load models
'''
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics


In [565]:
import pandas as pd

from pyspark.sql.functions import col

In [553]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv")

In [543]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [558]:
data.dtypes

age              int64
bloodpressure    int64
diabetes         int64
dtype: object

In [569]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
data['indexedLabel'] = label_encoder.fit_transform(data['age'])


In [573]:
train_data = data.sample(frac=0.7, random_state=42)
test_data = data.drop(train_data.index)


# **2. Decision Tree**
Run below codes and answer question 1.

reference:

model:
https://spark.apache.org/docs/latest/mllib-decision-tree.html

evaluation:
https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#multiclass-classification

## **Model**

In [608]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [616]:
data = pd.read_csv("dataset.csv")

In [617]:
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [624]:
ct = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), ["age", "bloodpressure", "diabetes"])
    ])


In [625]:
dt = DecisionTreeClassifier(random_state=42)

In [626]:
# Create the pipeline
dt_pipeline = Pipeline(steps=[("preprocessor", ct), ("classifier", dt)])

In [634]:
print(train_data.columns)


Index(['age', 'bloodpressure', 'diabetes'], dtype='object')


In [635]:
dt_pipeline.fit(train_data, train_data["age"])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'bloodpressure',
                                                   'diabetes'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

## **Model Evaluation**
You finish codes on the f1 and recall parts and run the code. Answer the question 1.

Accurancy

In [ ]:
acc_evaluator_dt = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy",)
acc_dt = acc_evaluator_dt.evaluate(dt_predictions)
print("accurancy:"+str(acc_dt))

Precision

In [ ]:
pr_evaluator_dt = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="precisionByLabel")
precision_dt = pr_evaluator_dt.evaluate(dt_predictions)
print("precision:"+str(precision_dt))

F1_score

Recall

## **3. Random forest Model**

## **Model**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(numTrees=3,featuresCol="indexedFeatures",labelCol="indexedLabel")

In [ ]:
rf_pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [ ]:
rf_model = rf_pipeline.fit(trainingData)

In [ ]:
rf_predictions = rf_model.transform(testData)

In [ ]:
print(rf_model.stages[2])

In [ ]:
rf_predictions.show(5)

## **Model Evaluation**
You finish codes on the precision and recall parts and run the code. Answer the question 2.

Accurancy

In [ ]:
acc_evaluator_rf = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy",)
acc_rf = acc_evaluator_rf.evaluate(rf_predictions)
print("accurancy:"+str(acc_rf))

F1_score

In [ ]:
f_evaluator_rf = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="f1")
f1_score_rf = f_evaluator_rf.evaluate(rf_predictions)
print("f1 score:"+str(f1_score_rf))

Precision

Recall

## **4. Naive Bayes Model**

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
import pandas as pd

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from pyspark.ml.feature import VectorAssembler


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset.csv")

In [ ]:
data['_c0'] = pd.to_numeric(data['age'])
data['_c1'] = pd.to_numeric(data['bloodpressure'])
data['_c2'] = pd.to_numeric(data['diabetes'])

In [ ]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values


In [ ]:
train_data = data.sample(frac=0.7, random_state=42)
test_data = data.drop(train_data.index)


In [ ]:
nb = MultinomialNB()

In [ ]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

In [ ]:
print(data.columns)


Index(['age', 'bloodpressure', 'diabetes', '_c0', '_c1'], dtype='object')


In [ ]:
model = GaussianNB()


In [ ]:
model.fit(train_data[['age', 'bloodpressure']], train_data['diabetes'])

GaussianNB()

In [ ]:
# Make predictions on the test data
predictions = model.predict(test_data[['age', 'bloodpressure']])

In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(test_data['diabetes'], predictions)
print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[2]]


## **Model Evaluation**
You finish codes on the precision and recall parts and run the code. Answer the question 2.

Accuracy

In [ ]:
accuracy = accuracy_score(test_data['diabetes'], predictions)
print(f"Model accuracy: {accuracy}")


Model accuracy: 1.0


Precision

In [ ]:
precision = precision_score(test_data['diabetes'], predictions, zero_division=1)
print(f"Model precision: {precision}")

Model precision: 1.0


F1_score

In [ ]:
f1 = f1_score(test_data['diabetes'], predictions, zero_division=1)
print(f"Model F1 score: {f1}")

Model F1 score: 1.0


Recall

In [ ]:
recall = recall_score(test_data['diabetes'], predictions, zero_division=1)
print(f"Model recall: {recall}")

Model recall: 1.0
